# f) Ngonye Falls Power and Energy Periods
Produces summary, agrregated results from the daily power and energy modelling.


## Inputs

| Data                       | Source                                        | Description                                 |
|----------------------------|-----------------------------------------------|---------------------------------------------|
| [Model Name]_pe_daily.csv | Notebook: e_power_energy | Daily power output  |


## Outputs
| File                           | Description                                 |
|--------------------------------|---------------------------------------------|
|    [Model Name]_pe_daily.csv ||
|    [Model Name]_pe_monthly.csv||
|    [Model Name]_pe_yearly.csv||
|    [Model Name]_pe_percentiles.csv||
|    [Model Name]_pe_monthly_day.csv||
|    [Model Name]_pe_daily_slim.csv||
|    [Model Name]_pe_weekly.csv||
|    [Model Name]_pe_weekly_slim.csv||
|    [Model Name]_pe_calmonthly.csv||
|    [Model Name]_pe_fdc.csv||
|    [Model Name]_pe_power_exceed.csv||
|    models.csv| |

## Libraries and Setup

In [71]:
import numpy as np
import pandas as pd
from scipy import interpolate
import math
import statsmodels.tsa.stattools as statools

## Parameters

In [72]:
input_data='./input_data/'
output_data='./output_data/2020/'

In [73]:
model_name='Base Case'#MW153#MW144 'MW162'#'Fixed Tailwater'#'Base Case' Headpond50cm EWRAllCs
dryrun=False #Dont write output files

In [74]:
#models=pd.read_csv(input_data + "models.csv").set_index('ModelName')
models=pd.read_csv(input_data + "2020/models.csv").set_index('ModelName')
model=models.loc[model_name]

out_prefix=model['OutputPrefix']#'base'
if out_prefix!='base':
    out_prefix="scenarios/" + out_prefix
    
plant_capacity=model['Capacity']#180
canal_capacity=plant_capacity/180*1100#1100

model

OutputPrefix                       base
Description                   Base Case
EWRCatSet              Recommendation 1
HeadpondLift                          0
Capacity                            180
FixedTailwater                        0
Mean                             830933
P95                              710170
P90                              751270
P75                              788598
P50                              838543
P25                              874970
P10                              920798
P5                               957532
StDev                             82123
CoefVar                       0.0988322
Max                         1.01916e+06
Min                              494343
CapFactor                      0.526975
CalYearMean                      831109
CalYearStDev                    78199.7
CalYearCoefVar                0.0940908
AutoCorrel                          NaN
Power_max                       193.908
CapFactor_Power_max             0.48928


## Load Daily Data

Load the daily time series which includes the exceedance values used for calculating daily EWRs.

In [75]:
daily=pd.read_csv(output_data + 'ngonye_daily' + '_base_pe.csv') #ngonye_daily_WC1_1_base_pe
daily=daily.set_index(pd.to_datetime(daily['Date'],format="%Y-%m-%d"))
daily=daily.drop('Date',axis=1)
daily.tail(5)

,LaggedDate,VicFalls,Conversion,Flow,Exceedance,Year,Month,Day,MonthId,WaterYear,...,EffGen2,EffGen3,EffGen4,PowerTurbine1,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy,PerfMwPerCumec
Date,,,,,,,,,,,,,,,,,,,,,
2020-05-16,2020-05-27,3453.1404,1.037022,3580.983662,0.056,2020,5,16,2020.05,2019,...,0.98,0.98,0.98,30.968655,30.968655,30.968655,30.968655,123.874619,2972.990860,0.034592
2020-05-17,2020-05-28,3438.1311,1.035186,3559.103900,0.057,2020,5,17,2020.05,2019,...,0.98,0.98,0.98,31.095199,31.095199,31.095199,31.095199,124.380797,2985.139121,0.034947
2020-05-18,2020-05-29,3393.3391,1.028636,3490.510716,0.059,2020,5,18,2020.05,2019,...,0.98,0.98,0.98,31.495004,31.495004,31.495004,31.495004,125.980014,3023.520337,0.036092
2020-05-19,2020-05-30,3304.8137,1.025935,3390.523841,0.063,2020,5,19,2020.05,2019,...,0.98,0.98,0.98,32.086290,32.086290,32.086290,32.086290,128.345160,3080.283848,0.037854
2020-05-20,2020-05-31,3217.6933,1.031443,3318.867319,0.070,2020,5,20,2020.05,2019,...,0.98,0.98,0.98,32.516351,32.516351,32.516351,32.516351,130.065405,3121.569722,0.039190


In [76]:
daily=daily.drop(daily.loc['2020-05-01':].index)
daily.tail(5)

,LaggedDate,VicFalls,Conversion,Flow,Exceedance,Year,Month,Day,MonthId,WaterYear,...,EffGen2,EffGen3,EffGen4,PowerTurbine1,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy,PerfMwPerCumec
Date,,,,,,,,,,,,,,,,,,,,,
2020-04-26,2020-05-07,4444.4855,1.029196,4574.246025,0.026,2020,4,26,2020.04,2019,...,0.98,0.98,0.98,25.688844,25.688844,25.688844,25.688844,102.755376,2466.129015,0.022464
2020-04-27,2020-05-08,4427.0109,1.024808,4536.836827,0.027,2020,4,27,2020.04,2019,...,0.98,0.98,0.98,25.872310,25.872310,25.872310,25.872310,103.489241,2483.741773,0.022811
2020-04-28,2020-05-09,4392.1861,1.023106,4493.670227,0.028,2020,4,28,2020.04,2019,...,0.98,0.98,0.98,26.085416,26.085416,26.085416,26.085416,104.341662,2504.199897,0.023220
2020-04-29,2020-05-10,4374.8357,1.023106,4475.918935,0.028,2020,4,29,2020.04,2019,...,0.98,0.98,0.98,26.173490,26.173490,26.173490,26.173490,104.693958,2512.655001,0.023390
2020-04-30,2020-05-11,4305.8472,1.018511,4385.553942,0.030,2020,4,30,2020.04,2019,...,0.98,0.98,0.98,26.625850,26.625850,26.625850,26.625850,106.503399,2556.081578,0.024285


## Monthly


In [77]:

monthly=daily.groupby("MonthId").mean().drop(['Day','EWRBandNo','EWRRefExceedance','Energy'],axis=1)

if 'VicFalls' in monthly.columns:
    monthly=monthly.drop(['VicFalls','Conversion','Volume','WaterDay','WaterWeek','Volume'],axis=1)

monthly=monthly.join(daily.groupby("MonthId").agg(    
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),   
   Turbines_max=('Turbines', 'max'),    
   Turbines_min=('Turbines', 'min'),   
   Energy=('Energy', 'sum'),
))
monthly['Days']=monthly.apply(lambda x: pd.Period(year=x['Year'],month=x['Month'],freq='D').days_in_month,axis=1)
monthly['CapFactor']=monthly['Energy']/(plant_capacity*24*monthly['Days'])

monthly.tail(12)

,Flow,Exceedance,Year,Month,WaterYear,WaterMonth,Flow_difference,EWRChannelA,EWRChannelC,EWRChannelD,...,PowerTurbine4,Power,PerfMwPerCumec,Power_max,Power_min,Turbines_max,Turbines_min,Energy,Days,CapFactor
MonthId,,,,,,,,,,,,,,,,,,,,,
2019.05,978.021034,0.337935,2019.0,5.0,2018.0,8.0,8.091851,42.000000,9.000000,8.000000,...,40.951327,159.059276,0.162640,167.592905,145.156890,4,3,118340.101240,31,0.883663
2019.06,721.987114,0.438067,2019.0,6.0,2018.0,9.0,10.811474,43.000000,9.666667,8.333333,...,NaN,116.263983,0.160200,148.460738,84.841276,3,2,83710.068068,30,0.645911
2019.07,435.845557,0.619774,2019.0,7.0,2018.0,10.0,7.114149,42.000000,9.000000,8.000000,...,NaN,64.385159,0.146620,90.532001,45.914520,2,1,47902.558566,31,0.357695
2019.08,282.944331,0.820097,2019.0,8.0,2018.0,11.0,3.644726,37.225806,7.000000,5.000000,...,NaN,38.478006,0.135595,50.475222,30.026529,1,1,28627.636746,31,0.213767
2019.09,191.608067,0.964833,2019.0,9.0,2018.0,12.0,2.340371,30.000000,6.000000,4.000000,...,NaN,25.744786,0.133251,33.737997,18.831016,1,1,18536.245770,30,0.143027
2019.10,148.031135,0.995903,2019.0,10.0,2019.0,1.0,3.397144,24.000000,4.000000,2.000000,...,NaN,20.224933,0.136214,25.585048,16.894071,1,1,15047.350157,31,0.112361
2019.11,210.314338,0.941800,2019.0,11.0,2019.0,2.0,2.627421,27.000000,4.000000,2.000000,...,NaN,32.651741,0.155264,39.932461,26.869203,1,1,23509.253427,30,0.181399
2019.12,308.201436,0.778258,2019.0,12.0,2019.0,3.0,4.811395,29.225806,4.645161,3.000000,...,NaN,51.918111,0.167167,70.469385,38.138518,2,1,38627.074674,31,0.288434
2020.01,516.568596,0.549194,2020.0,1.0,2019.0,4.0,8.443869,34.000000,5.000000,4.000000,...,NaN,89.858623,0.173427,114.075885,64.962330,3,2,66854.815566,31,0.499215


## Yearly

In [78]:
yearly=daily.groupby("WaterYear").agg(
   Flow_max=('Flow', 'max'),
   Flow_min=('Flow', 'min'), 
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_min=('EWRProportion', 'min'),
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_min=('HeadTurbine1', 'min'),    
   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_min=('LevelTailwater', 'min'),  
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),    
   Energy=('Energy', 'sum'),
)
yearly['CapFactor']=yearly['Energy']/(plant_capacity*24*365)


In [79]:
yearly=yearly.drop(yearly.loc['2019':].index)
yearly.tail(5)

,Flow_max,Flow_min,EWRProportion_max,EWRProportion_min,HeadTurbine1_max,HeadTurbine1_min,LevelTailwater_max,LevelTailwater_min,Power_max,Power_min,Energy,CapFactor
WaterYear,,,,,,,,,,,,
2014,1471.949472,214.231889,0.541252,0.100498,25.084232,18.578036,969.523298,964.709272,193.435679,26.767392,893812.599609,0.566852
2015,2992.244933,174.590793,0.497738,0.067253,25.390673,14.463909,973.599189,964.399465,190.989069,21.618629,792883.450415,0.502843
2016,3882.710344,154.808811,0.486115,0.047593,25.389864,12.656010,975.407088,964.399465,192.712079,18.260066,856528.211478,0.543207
2017,4867.713722,172.425625,0.551995,0.055970,25.390673,10.951475,977.111623,964.399465,189.075552,20.920096,843164.183046,0.534731
2018,1108.517240,162.535334,0.525139,0.109284,25.437708,20.382143,968.240946,964.399465,181.156847,18.275872,727576.735228,0.461426


In [80]:
yearly['Prop4Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==4,1,0))/365)
yearly['Prop3Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==3,1,0))/365)
yearly['Prop2Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==2,1,0))/365)
yearly['Prop1Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==1,1,0))/365)
yearly['Prop0Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==0,1,0))/365)
yearly['Days0Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==0,1,0)))


## Power and energy exceedance

In [81]:
power_exceed=pd.DataFrame(index=np.arange(0,1.05,0.05))
power_exceed.index=power_exceed.index.rename('Exceedance')
power_exceed['Power']=np.percentile(daily['Power'],(1-power_exceed.index)*100)
power_exceed.head(3)

,Power
Exceedance,
0.00,194.453700
0.05,182.541928
0.10,173.673585


In [82]:
edc=pd.DataFrame(index=np.arange(0,1.05,0.05))
edc['Energy']=np.percentile(yearly['Energy'],(1-edc.index)*100)

edc.head(3)

,Energy
0.00,1.026463e+06
0.05,9.649781e+05
0.10,9.284478e+05


In [83]:
yearly['Exceedance']=pd.merge_asof(yearly.reset_index().sort_values('Energy'),edc.reset_index().sort_values('Energy'),left_on='Energy',right_on='Energy').set_index('WaterYear')['index']

## Calendar years
Jan-Dec rather than Oct-Sept

In [84]:
min_year=daily['Year'].min()
max_year=daily['Year'].max()


In [85]:
calyearly=daily.groupby("Year").agg(  
   Energy=('Energy', 'sum'),
).drop([min_year,max_year],axis=0)
calyearly['CapFactor']=calyearly['Energy']/(180*24*365)
calyearly.head(5)

,Energy,CapFactor
Year,,
1925,861014.386973,0.546052
1926,785064.533826,0.497885
1927,857224.957899,0.543649
1928,834665.197694,0.529341
1929,764420.515594,0.484792


## Monthly Day
15th of every month

In [86]:
monthly_day=daily.loc[daily['Day']==15]

## Daily Slim
Slim has less columns and values are rounded so file size is reduced

In [87]:
daily_slim=daily.loc[:,
                     ['Year',
                      'Month',
                      'Day',
                      'WaterYear',
                      'Flow',
                      'EWRRefExceedance',
                      'EWRBandNo',
                      'EWRChannelA',
                      'EWRChannelC',
                      'EWRChannelD',
                      'EWRChannelE',
                      'EWRChannelFG',
                      'EWRTotal',
                      'EWRProportion',
                      'FlowCanal',
                      'FlowSpill',
                      'SpillChannelA',
                      'SpillChannelC',
                      'SpillChannelD',
                      'SpillChannelE',
                      'SpillChannelFG',
                      'FlowChannelA',
                      'FlowChannelC',
                      'FlowChannelD',
                      'FlowChannelE',
                      'FlowChannelFG',
                      'FlowLeftChannel',
                      'FlowTurbine1',
                      'FlowTurbine2',
                      'FlowTurbine3',
                      'FlowTurbine4',
                      'LevelHeadpond',
                      'LevelTailwater',
                      'HeadlossLeftChannel',
                      'HeadlossCanal',
                      'LevelForebay',
                      'Turbines',
                      'HeadTurbine1',
                      'HeadTurbine2',
                      'HeadTurbine3',
                      'HeadTurbine4',
                      'EffTurbine1',
                      'EffTurbine2',
                      'EffTurbine3',
                      'EffTurbine4',
                      'PowerTurbine1',
                      'PowerTurbine2',
                      'PowerTurbine3',
                      'PowerTurbine4',
                      'Power',
                      'Energy'
                     ]
                    ].fillna(0).astype({
                        'Flow': 'int32',
                        'FlowCanal': 'int32',
                        'FlowSpill': 'int32',
                        'EWRChannelA': 'int32',
                        'EWRChannelC': 'int32',
                        'EWRChannelD': 'int32',    
                        'EWRChannelE': 'int32',
                        'EWRChannelFG': 'int32',
                        'EWRTotal': 'int32',    
                        'SpillChannelA': 'int32',
                        'SpillChannelC': 'int32',
                        'SpillChannelD': 'int32',    
                        'SpillChannelE': 'int32',
                        'SpillChannelFG': 'int32',
                        'FlowChannelA': 'int32',
                        'FlowChannelC': 'int32',
                        'FlowChannelD': 'int32',    
                        'FlowChannelE': 'int32',
                        'FlowChannelFG': 'int32',
                        'FlowLeftChannel': 'int32',
                        'FlowTurbine1': 'int32',
                        'FlowTurbine2': 'int32',
                        'FlowTurbine3': 'int32',
                        'FlowTurbine4': 'int32',
                        'Energy': 'int32',
                        
                    }).round({
                            'EWRProportion': 2,
                            'LevelHeadpond': 1,
                            'LevelTailwater': 1,
                            'HeadlossLeftChannel': 2,
                            'HeadlossCanal': 2,
                            'LevelForebay': 1,
                            'PowerTurbine1': 1,
                            'PowerTurbine2': 1,
                            'PowerTurbine3': 1,
                            'PowerTurbine4': 1,
                            'Power': 1,
                            'HeadTurbine1':1,
                            'HeadTurbine2':1,
                            'HeadTurbine3':1,
                            'HeadTurbine4':1,
                            'EffTurbine1':2,
                            'EffTurbine2':2,
                            'EffTurbine3':2,
                            'EffTurbine4':2,    
                    })
daily_slim.tail(12)

,Year,Month,Day,WaterYear,Flow,EWRRefExceedance,EWRBandNo,EWRChannelA,EWRChannelC,EWRChannelD,...,EffTurbine1,EffTurbine2,EffTurbine3,EffTurbine4,PowerTurbine1,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy
Date,,,,,,,,,,,,,,,,,,,,,
2020-04-19,2020,4,19,2019,4592,0.25,2,70,14,18,...,0.85,0.85,0.85,0.85,25.6,25.6,25.6,25.6,102.4,2457
2020-04-20,2020,4,20,2019,4630,0.25,2,70,14,18,...,0.85,0.85,0.85,0.85,25.4,25.4,25.4,25.4,101.7,2439
2020-04-21,2020,4,21,2019,4692,0.13,2,70,14,18,...,0.85,0.85,0.85,0.85,25.1,25.1,25.1,25.1,100.5,2410
2020-04-22,2020,4,22,2019,4744,0.13,2,70,14,18,...,0.84,0.84,0.84,0.84,24.9,24.9,24.9,24.9,99.5,2387
2020-04-23,2020,4,23,2019,4692,0.13,2,70,14,18,...,0.85,0.85,0.85,0.85,25.1,25.1,25.1,25.1,100.5,2410
2020-04-24,2020,4,24,2019,4692,0.13,2,70,14,18,...,0.85,0.85,0.85,0.85,25.1,25.1,25.1,25.1,100.5,2410
2020-04-25,2020,4,25,2019,4592,0.13,2,70,14,18,...,0.85,0.85,0.85,0.85,25.6,25.6,25.6,25.6,102.4,2457
2020-04-26,2020,4,26,2019,4574,0.13,2,70,14,18,...,0.85,0.85,0.85,0.85,25.7,25.7,25.7,25.7,102.8,2466
2020-04-27,2020,4,27,2019,4536,0.13,2,70,14,18,...,0.85,0.85,0.85,0.85,25.9,25.9,25.9,25.9,103.5,2483


## Weekly & Weekly Slim
Slim has less columns and values are rounded so file size is reduced

In [88]:
weekly=daily.groupby(["WaterYear","WaterWeek"]).mean().drop(['Month','WaterMonth','Year','MonthId','Day','WaterDay','EWRBandNo','EWRRefExceedance','Turbines','Energy'],axis=1)

#if 'VicFalls' in weekly.columns:
weekly=weekly.drop(['VicFalls','Conversion','Volume'],axis=1)

weekly=weekly.join(daily.groupby(["WaterYear","WaterWeek"]).agg(    
   Flow_max=('Flow', 'max'),
   Flow_min=('Flow', 'min'), 
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_min=('EWRProportion', 'min'),
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_min=('HeadTurbine1', 'min'),    
   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_min=('LevelTailwater', 'min'),  
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),    
   Energy=('Energy', 'sum'),
))
weekly=weekly.join(daily[["WaterYear","WaterWeek","Flow"]].groupby(["WaterYear","WaterWeek"]).count().rename(columns={'Flow':'Count'}))
weekly['CapFactor']=weekly['Energy']/(180*24*weekly['Count'])
weekly=weekly.join(daily.reset_index('Date')[["WaterYear","WaterWeek","Date"]].groupby(["WaterYear","WaterWeek"]).first().rename(columns={'Date':'StartDate'}))

weekly.tail(12)

Flow  Exceedance  Flow_difference  EWRChannelA  \
WaterYear WaterWeek                                                          
2019      20          803.051219    0.403429         9.301837    36.000000   
          21          894.452169    0.368000        14.550082    36.000000   
          22         1194.773709    0.286286        67.003731    39.714286   
          23         2067.955671    0.171857       184.582275    49.000000   
          24         3381.701444    0.068714       182.951202    57.571429   
          25         4270.897857    0.033429        82.725837    59.000000   
          26         3979.529348    0.040571        47.523789    59.000000   
          27         3859.650309    0.044143        31.853763    68.428571   
          28         4069.505546    0.038143        40.038638    70.000000   
          29         4465.917825    0.028714        56.407324    70.000000   
          30         4646.684086    0.024857        46.114693    70.000000   
          31         4451.714368    0.028667        50.427628    70.000000   

                     EWRChannelC  EWRChannelD  EWRChannelE  EWRChannelFG  \
WaterYear WaterWeek                                                        
2019      20            6.000000     5.000000     4.000000     54.000000   
          21            6.000000     5.000000     4.000000     54.000000   
          22            6.857143     6.142857     5.428571     69.142857   
          23            9.000000     9.000000     9.000000    107.000000   
          24           10.714286    11.571429    12.428571    138.714286   
          25           11.000000    12.000000    13.000000    144.000000   
          26           11.000000    12.000000    13.000000    144.000000   
          27           13.571429    17.142857    23.285714    164.571429   
          28           14.000000    18.000000    25.000000    168.000000   
          29           14.000000    18.000000    25.000000    168.000000   
          30           14.000000    18.000000    25.000000    168.000000   
          31           14.000000    18.000000    25.000000    168.000000   

                       EWRTotal  EWRProportion  ...  HeadTurbine1_max  \
WaterYear WaterWeek                             ...                     
2019      20         105.000000       0.130834  ...         22.261264   
          21         105.000000       0.117508  ...         21.777988   
          22         127.285714       0.105367  ...         21.335759   
          23         183.000000       0.093933  ...         18.678636   
          24         231.000000       0.068949  ...         14.886225   
          25         239.000000       0.055982  ...         12.189918   
          26         239.000000       0.060079  ...         12.726527   
          27         287.000000       0.074361  ...         12.833420   
          28         295.000000       0.072528  ...         12.491120   
          29         295.000000       0.066107  ...         11.940971   
          30         295.000000       0.063501  ...         11.495688   
          31         295.000000       0.066274  ...         11.753518   

                     HeadTurbine1_min  LevelTailwater_max  LevelTailwater_min  \
WaterYear WaterWeek                                                             
2019      20                21.896978          967.160347          966.916979   
          21                21.307653          967.576509          967.234623   
          22                18.821812          969.304549          967.641902   
          23                15.134316          972.928782          969.428280   
          24                12.476824          975.586274          973.176873   
          25                11.783662          976.279436          975.873180   
          26                12.250510          975.812588          975.336571   
          27                12.521544          975.541554          975.229678   
          28                12.015549

In [89]:
weekly_slim=weekly.loc[:,
                     [
                      'StartDate',
                      'Flow',
                      'EWRChannelA',
                      'EWRChannelC',
                      'EWRChannelD',
                      'EWRChannelE',
                      'EWRChannelFG',
                      'EWRTotal',
                      'EWRProportion',
                      'FlowCanal',
                      'FlowSpill',
                      'SpillChannelA',
                      'SpillChannelC',
                      'SpillChannelD',
                      'SpillChannelE',
                      'SpillChannelFG',
                      'FlowChannelA',
                      'FlowChannelC',
                      'FlowChannelD',
                      'FlowChannelE',
                      'FlowChannelFG',
                      'FlowLeftChannel',
                      'FlowTurbine1',
                      'FlowTurbine2',
                      'FlowTurbine3',
                      'FlowTurbine4',
                      'LevelHeadpond',
                      'LevelTailwater',
                      'HeadlossLeftChannel',
                      'HeadlossCanal',
                      'LevelForebay',
                      'HeadTurbine1',
                      'HeadTurbine2',
                      'HeadTurbine3',
                      'HeadTurbine4',
                      'EffTurbine1',
                      'EffTurbine2',
                      'EffTurbine3',
                      'EffTurbine4',
                      'PowerTurbine1',
                      'PowerTurbine2',
                      'PowerTurbine3',
                      'PowerTurbine4',
                      'Power',
                      'Energy'
                     ]
                    ].fillna(0).astype({
                        'Flow': 'int32',
                        'FlowCanal': 'int32',
                        'FlowSpill': 'int32',
                        'EWRChannelA': 'int32',
                        'EWRChannelC': 'int32',
                        'EWRChannelD': 'int32',    
                        'EWRChannelE': 'int32',
                        'EWRChannelFG': 'int32',
                        'EWRTotal': 'int32',
                        'SpillChannelA': 'int32',
                        'SpillChannelC': 'int32',
                        'SpillChannelD': 'int32',    
                        'SpillChannelE': 'int32',
                        'SpillChannelFG': 'int32',
                        'FlowChannelA': 'int32',
                        'FlowChannelC': 'int32',
                        'FlowChannelD': 'int32',    
                        'FlowChannelE': 'int32',
                        'FlowChannelFG': 'int32',
                        'FlowLeftChannel': 'int32',
                        'FlowTurbine1': 'int32',
                        'FlowTurbine2': 'int32',
                        'FlowTurbine3': 'int32',
                        'FlowTurbine4': 'int32',
                        'Energy': 'int32',
                        
                    }).round({
                            'EWRProportion':2,
                            'LevelHeadpond': 1,
                            'LevelTailwater': 1,
                            'HeadlossLeftChannel': 2,
                            'HeadlossCanal': 2,
                            'LevelForebay': 1,
                            'PowerTurbine1': 1,
                            'PowerTurbine2': 1,
                            'PowerTurbine3': 1,
                            'PowerTurbine4': 1,
                            'Power': 1,
                            'HeadTurbine1':1,
                            'HeadTurbine2':1,
                            'HeadTurbine3':1,
                            'HeadTurbine4':1,
                            'EffTurbine1':2,
                            'EffTurbine2':2,
                            'EffTurbine3':2,
                            'EffTurbine4':2,
                    })

## Calendar Months

In [90]:
calmonthly=pd.DataFrame({'WaterMonth': [1,2,3,4,5,6,7,8,9,10,11,12],'MonthName': ['Oct','Nov','Dec','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep'], 'Month':[10,11,12,1,2,3,4,5,6,7,8,9]})
calmonthly=calmonthly.set_index('WaterMonth')
calmonthly=calmonthly.join(monthly.groupby(["WaterMonth"]).agg(    
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_mean=('EWRProportion', 'mean'),
   EWRProportion_median=('EWRProportion', 'median'),
   EWRProportion_min=('EWRProportion', 'min'),
    
    EWRTotal_max=('EWRTotal', 'max'),
   EWRTotal_mean=('EWRTotal', 'mean'),
   EWRTotal_median=('EWRTotal', 'median'),
   EWRTotal_min=('EWRTotal', 'min'),   
    
   FlowCanal_max=('FlowCanal', 'max'),
   FlowCanal_mean=('FlowCanal', 'mean'),
   FlowCanal_median=('FlowCanal', 'median'),
   FlowCanal_min=('FlowCanal', 'min'),     
    
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_mean=('HeadTurbine1', 'mean'),
   HeadTurbine1_median=('HeadTurbine1', 'median'),
   HeadTurbine1_min=('HeadTurbine1', 'min'), 

   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_mean=('LevelTailwater', 'mean'),    
   LevelTailwater_median=('LevelTailwater', 'median'),    
   LevelTailwater_min=('LevelTailwater', 'min'),  
   LevelHeadpond_max=('LevelHeadpond', 'max'),
   LevelHeadpond_mean=('LevelHeadpond', 'mean'),    
   LevelHeadpond_median=('LevelHeadpond', 'median'),    
   LevelHeadpond_min=('LevelHeadpond', 'min'),  
   Turbines_mean=('Turbines', 'mean'),      
   Power_max=('Power', 'max'),
   Power_mean=('Power', 'mean'),
   Power_median=('Power', 'median'),
   Power_min=('Power', 'min'),    
   Energy_max=('Energy', 'max'),
   Energy_mean=('Energy', 'mean'),
   Energy_median=('Energy', 'median'),    
   Energy_min=('Energy', 'min'),
))
calmonthly['Energy_P95']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.05)
calmonthly['Energy_P90']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.10)
calmonthly['Energy_P75']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.25)
calmonthly['Energy_P25']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.75)
calmonthly['Energy_P10']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.90)
calmonthly['Energy_P5']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.95)

calmonthly['Energy_stdev']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').std()
calmonthly['Energy_coefvar']=calmonthly['Energy_stdev']/calmonthly['Energy_mean']

calmonthly

,MonthName,Month,EWRProportion_max,EWRProportion_mean,EWRProportion_median,EWRProportion_min,EWRTotal_max,EWRTotal_mean,EWRTotal_median,EWRTotal_min,...,Energy_median,Energy_min,Energy_P95,Energy_P90,Energy_P75,Energy_P25,Energy_P10,Energy_P5,Energy_stdev,Energy_coefvar
WaterMonth,,,,,,,,,,,,,,,,,,,,,
1,Oct,10,0.474007,0.392678,0.393645,0.274079,120.0,93.669019,97.000000,52.0,...,21371.877684,9237.682821,16521.659151,17923.482582,19160.636043,27124.061891,31794.863869,36743.998976,6818.798016,0.287947
2,Nov,11,0.398357,0.330151,0.332595,0.215692,114.0,89.305556,93.000000,50.0,...,27187.980431,16239.971772,19570.052227,22120.769598,23433.170010,32231.432254,41154.493716,45618.717965,8923.893980,0.303759
3,Dec,12,0.287866,0.241694,0.242148,0.154067,126.0,100.264785,104.000000,59.0,...,47615.699954,28498.968077,36789.562428,38426.024714,42891.867360,60966.951948,69626.076712,86223.625623,14925.537515,0.284331
4,Jan,1,0.221174,0.176234,0.180704,0.096997,150.0,118.222110,120.000000,73.0,...,78705.666532,42300.999710,57116.818037,61995.576971,70339.189556,99118.075275,117328.986978,121516.257103,20632.801942,0.243301
5,Feb,2,0.187839,0.128346,0.137112,0.023941,197.0,139.262341,137.000000,85.0,...,100078.641346,6926.467251,73752.437737,78820.499991,90170.796657,111687.999854,119758.021776,122057.245627,18295.364118,0.184485
6,Mar,3,0.179652,0.094648,0.091091,0.040415,315.0,194.318548,201.064516,107.0,...,110834.665788,0.000000,20808.065263,77435.219698,95307.700562,121752.427934,131183.700245,133804.513695,30292.272758,0.291752
7,Apr,4,0.160351,0.084960,0.082360,0.050182,364.0,231.916667,217.000000,125.0,...,100054.804439,0.000000,61570.315818,71955.344038,82709.357450,115089.726073,129451.256809,134283.148206,26459.913806,0.269673
8,May,5,0.218251,0.130523,0.127747,0.065656,340.0,264.974873,276.000000,147.0,...,116935.208494,45661.618487,87312.169349,91444.656917,104378.933582,126468.769726,131096.531439,134219.455688,16499.646861,0.144778
9,Jun,6,0.366848,0.197286,0.192221,0.108821,293.0,212.694737,210.000000,119.0,...,114893.349588,32996.927634,51124.687182,72392.698900,96973.192779,121930.238720,123732.478909,125152.035046,23007.713111,0.217500


## Flow duration curve

In [91]:
daily['ExceedanceRnd']=np.round(daily['Exceedance']*2,2)/2
fdc=daily.reset_index().groupby('ExceedanceRnd').mean().drop(['VicFalls','Conversion','Year','Month','Day','MonthId','WaterYear','WaterDay','WaterMonth','WaterWeek'],axis=1)
fdc

,Flow,Exceedance,Volume,Flow_difference,EWRRefExceedance,EWRBandNo,EWRChannelA,EWRChannelC,EWRChannelD,EWRChannelE,...,EffGen2,EffGen3,EffGen4,PowerTurbine1,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy,PerfMwPerCumec
ExceedanceRnd,,,,,,,,,,,,,,,,,,,,,
0.000,8796.565576,0.001437,0.760023,140.531798,0.025775,1.098592,66.056338,12.408451,14.830986,17.183099,...,0.98,0.98,0.98,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.005,6422.509361,0.005011,0.554905,110.190545,0.042914,1.102857,72.371429,13.834286,17.674286,22.565714,...,0.98,0.98,0.98,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.010,5551.938987,0.010011,0.479688,66.928966,0.079375,1.301136,72.556818,13.607955,17.596591,22.937500,...,0.98,0.98,0.98,10.518905,10.518905,10.518905,10.518905,42.075620,1009.814891,0.007707
0.015,5228.737389,0.015083,0.451763,73.402759,0.102056,1.377778,72.038889,13.011111,16.811111,21.550000,...,0.98,0.98,0.98,22.728405,22.728405,22.728405,22.728405,90.913621,2181.926906,0.017399
0.020,4898.990451,0.020024,0.423273,95.385698,0.112202,1.416667,69.446429,13.160714,16.541667,21.107143,...,0.98,0.98,0.98,24.150753,24.150753,24.150753,24.150753,96.603013,2318.472305,0.019731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.980,180.974780,0.980197,0.015636,2.466468,0.893237,4.456647,28.560694,4.369942,2.803468,2.040462,...,NaN,NaN,NaN,24.689468,NaN,NaN,NaN,24.689468,592.547238,0.136405
0.985,173.033712,0.984892,0.014950,1.775355,0.908068,4.528409,27.431818,4.198864,2.517045,2.005682,...,NaN,NaN,NaN,23.702514,NaN,NaN,NaN,23.702514,568.860340,0.136994
0.990,164.598035,0.990241,0.014221,1.883200,0.944412,4.811765,25.917647,4.317647,2.429412,2.000000,...,NaN,NaN,NaN,22.747440,NaN,NaN,NaN,22.747440,545.938565,0.138172


## Add overall summary data to the model file

In [92]:

models.at[model_name,'Mean']=yearly['Energy'].mean()
models.at[model_name,'Max']=yearly['Energy'].max()
models.at[model_name,'Min']=yearly['Energy'].min()

models.at[model_name,'P95']=yearly['Energy'].quantile(0.05)
models.at[model_name,'P90']=yearly['Energy'].quantile(0.10)
models.at[model_name,'P75']=yearly['Energy'].quantile(0.25)
models.at[model_name,'P50']=yearly['Energy'].quantile(0.5)
models.at[model_name,'P25']=yearly['Energy'].quantile(0.75)
models.at[model_name,'P10']=yearly['Energy'].quantile(0.90)
models.at[model_name,'P5']=yearly['Energy'].quantile(0.95)

models.at[model_name,'StDev']=yearly['Energy'].std()
models.at[model_name,'CoefVar']=models.at[model_name,'StDev']/models.at[model_name,'Mean']

models.at[model_name,'CapFactor']=yearly['CapFactor'].mean()

models.at[model_name,'CalYearMean']=calyearly['Energy'].mean()
models.at[model_name,'CalYearStDev']=calyearly['Energy'].std()
models.at[model_name,'CalYearCoefVar']=models.at[model_name,'CalYearStDev']/models.at[model_name,'CalYearMean']

models.at[model_name,'Power_max']=yearly['Power_max'].max()
models.at[model_name,'CapFactor_Power_max']=calyearly['Energy'].mean()/(yearly['Power_max'].max()*24*365)

models.loc[model_name]



OutputPrefix                       base
Description                   Base Case
EWRCatSet              Recommendation 1
HeadpondLift                          0
Capacity                            180
FixedTailwater                        0
Mean                             838464
P95                              719835
P90                              758689
P75                              797362
P50                              843164
P25                              881920
P10                              928448
P5                               964978
StDev                           81402.6
CoefVar                       0.0970854
Max                         1.02646e+06
Min                              508216
CapFactor                      0.531751
CalYearMean                      838630
CalYearStDev                    77041.1
CalYearCoefVar                0.0918654
AutoCorrel                          NaN
Power_max                       194.454
CapFactor_Power_max            0.492323


In [93]:
autocorrel=pd.DataFrame(statools.acf(yearly['Energy'],fft=True,qstat=False,alpha=0.05))
autocorrel

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,1,0.470057,0.347898,0.288224,0.253285,0.160045,0.14524,0.0888819,0.176001,0.10438,...,-0.172877,-0.0924369,-0.00428005,0.0627289,0.0365915,-0.0284277,-0.000775512,0.00907542,-0.0187902,0.0197627
1,"[1.0, 1.0]","[0.2689692652222341, 0.6711453890231656]","[0.10643298639283033, 0.5893639225519856]","[0.02727603106162857, 0.549172108956372]","[-0.020233458688166728, 0.5268030514014819]","[-0.12279845731540212, 0.4428886820079799]","[-0.1412420435734652, 0.43172216704313954]","[-0.2005623269309781, 0.37832620293213925]","[-0.11454507792221211, 0.46654657636143426]","[-0.19044502285163434, 0.3992057734194334]",...,"[-0.538914919073664, 0.1931604332527414]","[-0.46176137524202715, 0.27688763247333587]","[-0.3745388942051406, 0.3659787977317553]","[-0.3075319637886479, 0.43298972937609914]","[-0.3340987831800034, 0.40728188015197975]","[-0.39926402951211976, 0.34240869064279295]","[-0.3716999808524608, 0.3701489576749676]","[-0.36184911872123887, 0.3799999509336451]","[-0.38972373666908616, 0.35214329044334364]","[-0.35120928062662043, 0.39073472115705604]"


## Save Files

In [95]:
outp=output_data+out_prefix

if dryrun==False:
    daily.to_csv(outp+'_pe_daily.csv')
    monthly.to_csv(outp+'_pe_monthly.csv')
    yearly.to_csv(outp+'_pe_yearly.csv')
    edc.to_csv(outp+'_pe_percentiles.csv')
    monthly_day.to_csv(outp+'_pe_monthly_day.csv')
    daily_slim.to_csv(outp+'_pe_daily_slim.csv')
    weekly.to_csv(outp+'_pe_weekly.csv')
    weekly_slim.to_csv(outp+'_pe_weekly_slim.csv')
    calmonthly.to_csv(outp+'_pe_calmonthly.csv')
    power_exceed.to_csv(outp+'_pe_power_exceed.csv')
    fdc.to_csv(outp+'_pe_fdc.csv')
    autocorrel.to_csv(outp+'_pe_autocorrel.csv')
    #models.to_csv(input_data + "models.csv")
    models.to_csv(input_data + "2020/models.csv")

In [96]:
years=daily['WaterYear'].unique().tolist()

for year in years:
    days=daily.loc[daily.WaterYear==year]
    days.to_csv(output_data + 'years/' + out_prefix + '_pe_daily_' + str(year) + '.csv')
 